In [ ]:
!pip install apscheduler
!pip install tzlocal==2.1
!pip install flask_cors
!pip install flask_ngrok
!pip install pyngrok
!pip install fuzzywuzzy
!pip install python-Levenshtein
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=c2b92ce4ff95601fab35979e319c29055ea5f9f319c2361cb260acebf5261dbc
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.8 MB/s eta 0:00:00


In [1]:
!ngrok authtoken some_key

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
from google.colab import files
import os
uploaded_files = list(files.upload().keys())
kaggle_json = uploaded_files[0]
new_name = 'kaggle.json'
os.rename(kaggle_json, new_name)
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d louisepatra/updatedmov
!kaggle kernels output louisepatra/notebookd572cc8cb7 -p ./output

Saving kaggle (4).json to kaggle (4).json
 99% 601M/605M [00:05<00:00, 150MB/s]
100% 605M/605M [00:05<00:00, 113MB/s]
Output file downloaded to ./output/data/dataset-metadata.json
Output file downloaded to ./output/data/updateddata.brotli.parquet
Output file downloaded to ./output/datafiles/rec1.npy
Output file downloaded to ./output/datafiles/rec2.npy
Output file downloaded to ./output/imdb.tsv
Output file downloaded to ./output/indices.pic
Output file downloaded to ./output/revindices.pic
Output file downloaded to ./output/title.ratings.tsv.gz
Kernel log downloaded to ./output/notebookd572cc8cb7.log 


In [8]:
!unzip /content/updatedmov.zip
!unzip /content/tmdb-dataset.zip

Archive:  /content/updatedmov.zip
  inflating: updateddata.brotli.parquet  
Archive:  /content/tmdb-dataset.zip
  inflating: TMDB_Dataset.brotli.parquet  


In [9]:
!cp /content/TMDB_Dataset.brotli.parquet /content/temp.brotli.parquet
!cp -r /content/drive/MyDrive/files /content/files
!cp /content/output/datafiles/rec1.npy /content/rec1.npy
!cp /content/output/datafiles/rec2.npy /content/rec2.npy
!cp /content/output/indices.pic /content/indices.pic
!cp /content/output/revindices.pic /content/revindices.pic

In [10]:
#Parent Server
from click import echo
from apscheduler.schedulers.background import BackgroundScheduler
from flask import Flask, jsonify,request,send_from_directory
from flask_ngrok import run_with_ngrok as rn
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from time import time
import pandas as pd
import glob
import requests
from flask_cors import CORS
import numpy as np
from json import dumps
from urllib.parse import urlparse
import gc
from fuzzywuzzy import fuzz

d=pd.DataFrame()
userwd={}
userwl={}
rec={}
buffer={}
ord={"desc":False,"asc":True,"":False," ":False}
code={'Unknown': 1,
 'Action': 2,
 'Adventure': 4,
 'Animation': 8,
 'Comedy': 16,
 'Crime': 32,
 'Documentary': 64,
 'Drama': 128,
 'Family': 256,
 'Fantasy': 512,
 'History': 1024,
 'Horror': 2048,
 'Music': 4096,
 'Mystery': 8192,
 'Romance': 16384,
 'Science Fiction': 32768,
 'TV Movie': 65536,
 'Thriller': 131072,
 'War': 262144,
 'Western': 524288}
c=0
cc=0
dsaved=None
so=None
papiip=capiip=None
indices=q_movies=cosine_sim=cosine_sim2=m2=C2=revindices=lcm=None
def myapiip():
  global papiip
  papiip= requests.get("https://api.ngrok.com/endpoints", headers={
    "Authorization": "Bearer 2efwsf",
    "Ngrok-Version": "2"
  }).json()['endpoints'][0]['public_url']
def sortorder(s):
    global c
    if s.lower()=="date" and c!=0:
        d.sort_values(by='release_date',ascending=False,inplace=True)
        c=0
    elif s.lower()=="rating" and c!=1:
        d.sort_values(by='vote_average',ascending=False,inplace=True)
        c=1
    elif s.lower()=="title" and c!=2:
        d.sort_values(by='title',ascending=False,inplace=True)
        c=2
    elif s.lower()=="popularity" and c!=3:
        d.sort_values(by='imdbvote',ascending=False,inplace=True)
        c=3
def init():

    global indices,q_movies,cosine_sim,cosine_sim2,m2,C2,revindices,lcm
    global d,dsaved
    d=pd.read_parquet("temp.brotli.parquet")
    print("Succesfully Loaded Data..")
    d.replace(to_replace = np.nan, value =" ",inplace=True)
    dsaved=d.copy()
    print(gc.collect())
    lcm=d[['vote_average','vote_count','title','overview','soup']]
    # print("Loaded")
    C2= lcm['vote_average'].mean()
    m2= lcm['vote_count'].quantile(0.98)
    m2=80
    print(m2,C2)
    q_movies = lcm[lcm['vote_count'] >= m2]
    q_movies['score'] = q_movies.apply(weighted_rating, args=(m2,C2),axis=1)
    q_movies = q_movies.sort_values('score', ascending=False)
    a=q_movies[['score']]
    # del q_movies
    q_movies=a
    print("Applied filter.")
    # lcm=lcm[(lcm['vote_count'] >m2)].reset_index()
    # indices = pd.Series(lcm.index, lcm['id'])
    # revindices=pd.Series(lcm['id'],lcm.index)
    # lcm.set_index(['id'],inplace=True)
    # # print("len after",len(lcm))
    # print(gc.collect())
    # tfidf = TfidfVectorizer(stop_words='english')
    # tfidf_matrix = tfidf.fit_transform(lcm['overview']).astype(np.float16)
    # cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    # print("Linear Kernel 1 Finished")
    # print(gc.collect())
    # count = CountVectorizer(stop_words='english')
    # # tfidf_matrix = tfidf.fit_transform(lcm['overview']).astype(np.float16)
    # count_matrix = count.fit_transform(lcm['soup']).astype(np.float16)
    # cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
    # print("Cosine Sim 2 Finished")
    print(gc.collect())
    files=['userwd.pi','userwl.pi','rec.pi','buffer.pi']
    if files[0] in (glob.glob("*.pi")):
        userwd=pickle.load(open(files[0],"rb"))
    if files[1] in (glob.glob("*.pi")):
        userwl=pickle.load(open(files[1],"rb"))
    if files[2] in (glob.glob("*.pi")):
        rec=pickle.load(open(files[2],"rb"))
    if files[3] in (glob.glob("*.pi")):
        buffer=pickle.load(open(files[3],"rb"))
    print("Loaded pickle data.")
    global rec1,rec2
    rec1=np.load('rec1.npy')
    rec2=np.load('rec2.npy')
    print(rec1[0])
    print(rec1[1])
    # ind=pd.read_csv("indices.csv").set_index('0')
    revindices= pickle.load(open("revindices.pic", "rb"))
    indices= pickle.load(open("indices.pic", "rb"))
    print("Loaded recommed files")

    # display(d)
def saved():
    print("Started saving")
    files=['userwd.pi','userwl.pi','rec.pi','buffer.pi']
    with open(files[0],'wb') as f:
        pickle.dump(userwd,f)
    with open(files[1],'wb') as f:
        pickle.dump(userwl,f)
    with open(files[2],'wb') as f:
        pickle.dump(rec,f)
    with open(files[3],'wb') as f:
        pickle.dump(buffer,f)
    print("Done Saving.")

sched = BackgroundScheduler(daemon=True)
sched.add_job(saved,'interval',minutes=100)
sched.start()

app = Flask(__name__)
CORS(app)
rn(app)
# @app.route('/uploads/<path:filename>')
# def download_file(filename):
#     return send_from_directory(app.config['UPLOAD_FOLDER'],
#                                filename, as_attachment=True)

@app.route('/files/<path:filename>')
def download_file(filename):
    print(filename)
    return send_from_directory("./files/",filename, as_attachment=False)

def calculate_similarity(search_string, cell_string):
  return(fuzz.partial_ratio( search_string,cell_string))


def searchorder(sstr):
    print(sstr)
    global so,dsaved
    if sstr is None:
        d.sort_values(by='release_date',ascending=False,inplace=True)
        c=0
        print("1")
        so=None
        return(dsaved.copy())
    elif sstr!=so:
        string=sstr
        col='title'
        mask = d[col].str.contains(string, case=False)
        d['similarity'] = d[mask][col].apply(calculate_similarity, args=(string,))
        df_sorted = d.sort_values(by='similarity', ascending=False)
        df_selected = df_sorted[mask]
        so=sstr
        print("2")
        return(df_selected)
    else:
        print("3")
        return(d)



@app.route("/connection",methods=['GET'])
def connect():
    global connect,capiip,cc
    print(request.args)
    capiip=request.args['child']
    print(capiip)
    print(cc==0)
    if cc==0:
        cc=1
        return(jsonify({" ":" "}))
    return(jsonify({"msg":"hello kid"}))

@app.route("/api/movies",methods=['GET'])
def get_movies():
    global d
    print(request.args)
    if "start" in request.args and "end" in request.args:
        # return("hello")
        sstr=request.args['search'] if 'search' in request.args else None
        if sstr=="" or sstr==" ":sstr=None
        d=searchorder(sstr)
        o=urlparse(request.base_url).netloc
        start=int(request.args['start'])
        end=min(len(d),int(request.args['end']))
        sortby=request.args['sortby'] if 'sortby' in request.args else "date"
        if sortby==" " or sortby=="":
          sortby="date"
        orderby=ord[request.args['orderby'] if 'orderby' in request.args else "desc"]
        # print(request.args['genres'].split(','))
        # print("genres" in request.args)
        g=request.args['genres'].split(',') if "genres" in request.args else 0
        if g==[" "] or g==[""]:
          g=0
        s=0
        if 'dir' in request.args:
            dir=request.args['dir']
            if dir=="forward":
                start+=1
            elif dir=="backward":
                end-=1
        print(g)
        if g:
            for i in g:s+=code[i]
        # print(start,end)
        t=[]
        p={"page":"none"}
        c=start
        # print(i,g)
        sortorder(sortby)
        i=start
        if orderby==False:
            while(i<len(d) and c<end):
                # print(i,c)
                if g and int(d.iloc[i].gencode)&s==s:
                    c+=1
                    k={}
                    k["gen"]=str(d.iloc[i].genres)
                    # k["gen"]=g[:]
                    k['id']=str(d.index.values[i])
                    if d.iloc[i]['poster_path']==" ":
                        k['poster']="https://"+o+"/files/poster.png"
                    else:
                        k['poster']="https://image.tmdb.org/t/p/original"+d.iloc[i]['poster_path']
                    k['title']=str(d.iloc[i]['original_title'])
                    k['year']=str(d.iloc[i]['release_date'])
                    k['rating']=str(d.iloc[i]['vote_average'])
                    k['loc']=i
                    t.append(k)
                elif g==0:
                    c+=1
                    k={}
                    k['id']=str(d.index.values[i])
                    k["gen"]=str(d.iloc[i].genres)
                    if d.iloc[i]['poster_path']==" ":
                        k['poster']="https://"+o+"/files/poster.png"
                    else:
                        k['poster']="https://image.tmdb.org/t/p/original"+d.iloc[i]['poster_path']
                    k['title']=str(d.iloc[i]['original_title'])
                    k['year']=str(d.iloc[i]['release_date'])
                    k['rating']=str(d.iloc[i]['vote_average'])
                    k['loc']=i
                    t.append(k)
                i+=1
            if i==len(d):
              p['page']="end"
            elif start<1:
              p['page']='start'
        else:
            end=len(d)-1-end
            start=len(d)-1-start
            i=start
            c=start
            print(start,end,i,c)
            while(i>=0 and c>=end):
                # print(i,c)
                if (g==0) or (g and int(d.iloc[i].gencode)&s==s):
                    c-=1
                    k={}
                    # k["gen"]=g[:]
                    k["gen"]=str(d.iloc[i].genres)
                    k['id']=str(d.index.values[i])
                    if d.iloc[i]['poster_path']==" ":
                        k['poster']="https://"+o+"/files/poster.png"
                    else:
                        k['poster']="https://image.tmdb.org/t/p/original"+d.iloc[i]['poster_path']
                    k['title']=str(d.iloc[i]['original_title'])
                    k['year']=str(d.iloc[i]['release_date'])
                    k['rating']=str(d.iloc[i]['vote_average'])
                    k['loc']=len(d)-1-i
                    t.append(k)
                # elif g==0:
                #     c-=1
                #     k={}
                #     k['id']=str(d.index.values[i])
                #     k["gen"]=str(d.iloc[i].genres)
                #     if d.iloc[i]['poster_path']==" ":
                #         k['poster']="https://"+o+"/files/poster.png"
                #     else:
                #         k['poster']="https://image.tmdb.org/t/p/original"+d.iloc[i]['poster_path']
                #     k['title']=str(d.iloc[i]['original_title'])
                #     k['year']=str(d.iloc[i]['release_date'])
                #     k['rating']=str(d.iloc[i]['vote_average'])
                #     k['loc']=len(d)-1-i
                #     t.append(k)
                i-=1
            print(i,i>=len(d)-1,i==0)
            print(start<=0,i<=0,end>=len(d)-1,i>=len(d)-1)
            if i<=0:
              p['page']="end"
            elif start>=len(d)-1:
              p['page']="start"
        t.append(p)
        print("Done")
        return(jsonify(t))

def getmovie(idu=None):
    k={}
    o=urlparse(request.base_url).netloc
    id=int(idu)
    # print("id",id)
    # print(len())
    # print(id in d.index,str(id) in d.index)
    if id in d.index:
        k['id']=id
        if d.loc[id]['poster_path']==" ":
            k['poster']="https://"+o+"/files/poster.png"
        else:
            k['poster']="https://image.tmdb.org/t/p/original"+d.loc[id]['poster_path']
        k['title']=d.loc[id]['original_title']
        k['year']=str(d.loc[id]['release_date'])
        k['rating']=d.loc[id]['vote_average']
    else:
        return("")
    # print(k)
    return(k)

@app.route("/api/movie",methods=['GET'])
def get_movie():
    print(request.args)
    o=urlparse(request.base_url).netloc
    a=""
    c=""
    k={}
    if 'id' in request.args:
        id = int(request.args['id'])
    else:
        return("Error:No id is there")
    print(d.index[0])
    print(type(d.index[0]))
    if id in d.index:
        k['id']=id
        if d.loc[id]['poster_path']==" ":
            k['poster']="https://"+o+"/files/poster.png"
        else:
            k['poster']="https://image.tmdb.org/t/p/original"+d.loc[id]['poster_path']
        k['title']=d.loc[id]['original_title']
        k['year']=d.loc[id]['release_date']
        k['rating']=str(d.loc[id]['vote_average'])
        k['back']=str(d.loc[id]['backdrop_path'])
        if d.loc[id]['backdrop_path']==" ":
            k['back']="https://"+o+"/files/baground.jpg"
        else:
            k['back']="https://image.tmdb.org/t/p/original"+d.loc[id]['backdrop_path']
        print(d.loc[id]['belongs_to_collection'])
        e=eval(d.loc[id]['belongs_to_collection'])
        if e!=" " and not e is None:

            print(e)
            k['collection']=e["name"]
        else:
            k['collection']="None"
        k['lang']=d.loc[id]['original_language']
        k['len']=str(d.loc[id]['runtime'])
        k['status']=d.loc[id]['status']
        k['imdbrat']=str(d.loc[id]['imdbrat'])
        k['imdbvote']=str(d.loc[id]['imdbvote'])
        k['tmdbvote']=str(d.loc[id]['vote_count'])
        k['genres']=str(d.loc[id]['genres'])
        k['overview']=str(d.loc[id]['overview'])
        i=eval(d.loc[id].cast)
        name=list()
        char=list()
        pop=list()
        a=[]
        c=[]

        if not i is None:
            for j in range(0,len(i)):
                b={}
                b['name']=i[j]['name']
                b['character']=i[j]['character']
                b['popularity']=i[j]['popularity']
                if i[j]['profile_path'] is not None:
                    b['pic']="https://image.tmdb.org/t/p/original"+i[j]['profile_path']
                else:
                    b['pic']="https://"+o+"/files/"+str(i[j]['gender'])+".png"
                a.append(b)
            a=sorted(a,key=lambda x:x['popularity'],reverse=True)
        j=eval(d.loc[id]['crew'])
        if not j is None:
            for z in range(0,len(j)):
                b={}
                b['name']=j[z]['name']
                b['job']=j[z]['job']
                if "director" in b['job'].lower():
                    b['pop']=1
                elif "producer" in b['job'].lower():
                    b['pop']=2
                else:
                    b['pop']=3
                if j[z]['profile_path'] is not None:
                    b['pic']="https://image.tmdb.org/t/p/original"+j[z]['profile_path']
                else:
                    b['pic']="https://"+o+"/files/"+str(j[z]['gender'])+".png"
                c.append(b)
            c=sorted(c,key=lambda x:x['pop'])
        k['cast']=a
        k['crew']=c
    else:
        return("Wrong ID")
    print(k)
    return jsonify(k)

@app.route("/temp",methods=['GET'])
def temp():
    return(str(userwd)+" "+str(userwl))


@app.route("/api/initrec",methods=['GET'])
def initrec():

    return("done")


def weighted_rating(x, m, C):
    global m2,C2,q_movies,indices,cosine_sim,cosine_sim2
    # print(m,C)
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
def recommendtop(n=10):
    global m2,C2,q_movies,indices,cosine_sim,cosine_sim2
    return set(q_movies.index[:n])
def get_recommendations2(id ,n,cosine_sim2,indices,revindices):
    # print("rec",d.loc[id].title)
    idx = indices[id]
    sim_scores = sorted(list(enumerate(cosine_sim2[idx])), key=lambda x: x[1], reverse=True)[1:n+1]
    movie_indices = [revindices[i[0]] for i in sim_scores]
    # for i in movie_indices:
    #   print(d.loc[i].title)
    # print("done")
    return set(movie_indices)

def get_recommendations(id ,n,cosine_sim2,indices,revindices,c=1):
    # print("rec",d.loc[id].title)
    if id not in indices:
      return(set())
    idx = indices[id]
    # print(idx,id)
    # sim_scores = sorted(list(enumerate(cosine_sim2[idx])), key=lambda x: x[1], reverse=True)[1:n+1]
    if c==1:
      movie_indices = [revindices[i] for i in rec1[idx][1:n+1]]
    else:
      movie_indices = [revindices[i] for i in rec2[idx][1:n+1]]
    # for i in movie_indices:
    #   print(d.loc[i].title)
    # print("done")
    return set(movie_indices)

@app.route("/api",methods=['GET'])
def evalt():
    print(request.args)
    s=request.args['eval']
    print(s)
    k=str(eval(s))
    print(eval(s))
    return(k)
@app.route("/user/recommend",methods=['GET'])
def getrecommend():
    global indices,revindices
    print(request.args)
    # print(rec)
    # print(revindices)
    if "userid" in request.args:
        uid=(request.args['userid'])
        if uid in buffer:
            s=set(recommendtop(10))
            for i in list(buffer[uid]):
                # print(i,buffer)
                # print(i in lcm.index)
                if i in d.index:
                    # print("i",i)
                    # print("set",s)
                    s.update(get_recommendations(i,10,cosine_sim,indices,revindices,1))
                    s.update(get_recommendations(i,10,cosine_sim2,indices,revindices,2))
                    # print("after set",s)
            # for i in s:
            #   print("det",d.loc[i].title)
            if uid in userwd:
                s=s.difference(userwd[uid])
                userwd[uid].update(buffer[uid])
                buffer[uid]=set()
            else:
                userwd[uid]=(buffer[uid])
                buffer[uid]=set()
            if uid in userwl:
                s=s.difference(userwl[uid])
            if uid in rec:
                rec[uid].update(s)
                rec[uid]=rec[uid].difference(userwd[uid])
            else:
                rec[uid]=s
            k=[]
            l=0
            for i in rec[uid]:
                # print("iii",i)
                c=getmovie(idu=i)
                if len (c)>3:
                    k.append(c)
            # print(list(k))
            return(dumps(k))
        else:
            return "Error:Wrong Format - No inner id found"
    else:
        return "Error:Wrong Format - No User ID"

@app.route("/user",methods=['GET'])
def update_list():
    print(request.args)
    if "userid" in request.args:
        uid=(request.args['userid'])
        if 'addwatched' in request.args:
            movid=int(request.args['addwatched'])
            # if uid in userwd:
            #     userwd[uid].add(movid)
            # else:
            #     userwd[uid]={movid}
            if uid in buffer:
                buffer[uid].add(movid)
            else:
                buffer[uid]={movid}
        elif 'removewatched' in request.args:
            movid=int(request.args['removewatched'])
            if uid in userwd:
                userwd[uid].remove(movid)
            else:
                return("User id not exists")
            if uid in buffer:
                buffer[uid].remove(movid)
        elif 'addwatchlist' in request.args:
            movid=int(request.args['addwatchlist'])
            if uid in userwl:
                userwd[uid].add(movid)
            else:
                userwd[uid]={movid}
        elif 'removewatchlist' in request.args:
            movid=int(request.args['removewatchlist'])
            if uid in userwl:
                userwl[uid].remove(movid)
            else:
                return("User id not exists")
        else:
            return "Error:Wrong Format - No inner if found"
    else:
        return "Error:Wrong Format - No User ID"
    return("Done")
@app.route("/api/credits",methods=['GET'])
def get_credits():
    print(request.args)
    k={}
    if 'id' in request.args:
        id = int(request.args['id'])
    else:
        return("Error:No id is there")
    if id in d.index:
        k['id']=id
        k['name']=d.loc[id]["cast"][0]["name"]
        k['character']=d.loc[id]["cast"][0]["character"]
        k['popularity']=d.loc[id]["cast"][0]["popularity"]
    else:
        return("Wrong ID")
    print(k)
    return jsonify(k)
@app.route("/api/save",methods=['GET'])
def save():
    print(request.args)
    o = urlparse(request.base_url)
    print(o.netloc)
    print()
    global d
    if "show" in request.args:
        # return("hello")
        return(str(glob.glob("*.pi")))
    files=['userwd.pi','userwl.pi','rec.pi','buffer.pi']
    with open(files[0],'wb') as f:
        pickle.dump(userwd,f)
    with open(files[1],'wb') as f:
        pickle.dump(userwl,f)
    with open(files[2],'wb') as f:
        pickle.dump(rec,f)
    with open(files[3],'wb') as f:
        pickle.dump(buffer,f)
    return("done saving")

@app.route("/api/load",methods=['GET'])
def load():
    print(request.args)
    global d,userwd,userwl,rec,buffer
    files=['userwd.pi','userwl.pi','rec.pi','buffer.pi']
    if files[0] in (glob.glob("*.pi")):
        userwd=pickle.load(open(files[0],"rb"))
    if files[1] in (glob.glob("*.pi")):
        userwl=pickle.load(open(files[1],"rb"))
    if files[2] in (glob.glob("*.pi")):
        rec=pickle.load(open(files[2],"rb"))
    if files[3] in (glob.glob("*.pi")):
        buffer=pickle.load(open(files[3],"rb"))
    # print(userwd)
    # print(userwl)
    # print(buffer)
    # print(rec)
    return("Done")
init()
app.run()

Succesfully Loaded Data..
0
80 2.368603265387161
Applied filter.
0
Loaded pickle data.
[22548  2028 22093  9986 11399 19212 19122  2725  6836  9858  4605 24270
 10913  4984 22615 26329  6514  9163  7204 10555 13689  6020  3644  1599
 13309  7848 13307 15050  7691 14079]
[19434  6937 21676 21330  7753  7452 11665 17192  4149  7173 15831   909
 20883 25547 15722 15268 21351  7281   361 16078  4197 15427 11373 23944
  4454 24957 25270 23602 10492    38]
Loaded recommed files
 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-10-c923a51d3e56>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_movies['score'] = q_movies.apply(weighted_rating, args=(m2,C2),axis=1)
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8308-35-237-112-217.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
ImmutableMultiDict([('start', '0'), ('end', '20')])
None
1


INFO:werkzeug:127.0.0.1 - - [25/Jul/2023 16:50:07] "GET /api/movies?start=0&end=20 HTTP/1.1" 200 -


0
Done
ImmutableMultiDict([('start', '0'), ('end', '20'), ('sortby', 'Date'), ('orderby', 'desc')])
None
1


...

ImmutableMultiDict([('id', '271240')])
271240
<class 'numpy.int64'>

ImmutableMultiDict([('start', '0'), ('end', '20')])
None
1


INFO:werkzeug:127.0.0.1 - - [25/Jul/2023 17:03:41] "GET /api/movies?start=0&end=20 HTTP/1.1" 200 -


0
Done


ERROR:__main__:Exception on /api/movie [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask_cors/extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-10-c923a51d3e56>", line 377, in get_movie
    e=eval(d.loc[id]['belongs_to_collection'])
  File "<string>", line 0
    
SyntaxError: invalid syntax
INFO:

ImmutableMultiDict([('id', '271251')])
271240
<class 'numpy.int64'>



In [ ]:
rec2[155]

array([2124, 2347, 2933, 3183, 3207, 3291, 3470, 3515, 3602, 3705, 3752,
       3785, 3787, 3789, 3808, 3829, 3971, 3974, 4034, 4056, 4081, 4091,
       4120, 4127, 4138, 4148, 4157, 4176, 4244, 4277], dtype=uint32)

In [ ]:
ind=pd.read_csv("indices.csv").set_index('0')

In [ ]:
revindices={}
indices={}
for i,j in ind.to_dict()['id'].items():
  revindices[j]=i
  indices[i]=j
rev

Streaming output truncated to the last 5000 lines.
20092 10117
20093 9717
20094 17126
20095 21733
20096 2093
20097 10644
20098 31701
20099 11503
20100 28452
20101 12714
20102 4271
20103 62087
20104 11822
20105 27390
20106 27352
20107 26355
20108 24929
20109 23752
20110 28848
20111 34870
20112 8463
20113 22572
20114 31056
20115 41952
20116 32148
20117 43680
20118 46188
20119 57918
20120 801
20121 11174
20122 13246
20123 20115
20124 62491
20125 22477
20126 40962
20127 11548
20128 24828
20129 18305
20130 13247
20131 10780
20132 12626
20133 2039
20134 22500
20135 11896
20136 10673
20137 10110
20138 48309
20139 28448
20140 11513
20141 9964
20142 12154
20143 2609
20144 17159
20145 28370
20146 15582
20147 40634
20148 15658
20149 53653
20150 58613
20151 3543
20152 4885
20153 42002
20154 15785
20155 27786
20156 13703
20157 42010
20158 12506
20159 26677
20160 35151
20161 42012
20162 5753
20163 28218
20164 11496
20165 12476
20166 27764
20167 35463
20168 40461
20169 34379
20170 12623
20171 8852
20

{774752: 0,
 821881: 1,
 736918: 2,
 661374: 3,
 791177: 4,
 593643: 5,
 802372: 6,
 936960: 7,
 338958: 8,
 1013860: 9,
 632856: 10,
 948276: 11,
 833097: 12,
 505642: 13,
 668461: 14,
 1022256: 15,
 896485: 16,
 829280: 17,
 823766: 18,
 892515: 19,
 944864: 20,
 901563: 21,
 979924: 22,
 615952: 23,
 595586: 24,
 930921: 25,
 962232: 26,
 873125: 27,
 788977: 28,
 675054: 29,
 1032950: 30,
 732459: 31,
 1039026: 32,
 676547: 33,
 674324: 34,
 511817: 35,
 744114: 36,
 886083: 37,
 541134: 38,
 436270: 39,
 779782: 40,
 886396: 41,
 1040330: 42,
 869025: 43,
 864959: 44,
 919573: 45,
 759507: 46,
 616820: 47,
 1022206: 48,
 826769: 49,
 1024530: 50,
 1007401: 51,
 830784: 52,
 817758: 53,
 49046: 54,
 848058: 55,
 999722: 56,
 1018648: 57,
 901385: 58,
 663712: 59,
 851644: 60,
 848791: 61,
 335795: 62,
 884363: 63,
 723419: 64,
 823951: 65,
 1015724: 66,
 1023086: 67,
 614939: 68,
 597922: 69,
 799546: 70,
 816952: 71,
 714888: 72,
 797840: 73,
 1027014: 74,
 338947: 75,
 664469: 76

In [ ]:
indices.to_dict().keys()

dict_keys(['id'])

In [ ]:
import pandas as pd
import numpy as np
d=pd.read_csv("/content/drive/MyDrive/finalupdated.csv")
d=d.rename({"Unnamed: 0":"id"},axis=1)

In [ ]:
d

,id,movie,credits,watch,keys
0,271240,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 271240, 'cast': [{'adult': False, 'gend...","{'id': 271240, 'results': {}}","{'id': 271240, 'keywords': []}"
1,271241,"{'adult': False, 'backdrop_path': '/fuJGtM9YQC...","{'id': 271241, 'cast': [{'adult': False, 'gend...","{'id': 271241, 'results': {'AU': {'link': 'htt...","{'id': 271241, 'keywords': []}"
2,271242,"{'adult': False, 'backdrop_path': '/pXHyPNJM5h...","{'id': 271242, 'cast': [{'adult': False, 'gend...","{'id': 271242, 'results': {'CA': {'link': 'htt...","{'id': 271242, 'keywords': []}"
3,271243,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 271243, 'cast': [], 'crew': []}","{'id': 271243, 'results': {}}","{'id': 271243, 'keywords': []}"
4,271244,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 271244, 'cast': [], 'crew': []}","{'id': 271244, 'results': {}}","{'id': 271244, 'keywords': []}"
...,...,...,...,...,...
805481,346369,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 346369, 'cast': [{'adult': False, 'gend...","{'id': 346369, 'results': {}}","{'id': 346369, 'keywords': []}"
805482,346371,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 346371, 'cast': [{'adult': False, 'gend...","{'id': 346371, 'results': {}}","{'id': 346371, 'keywords': []}"
805483,346372,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 346372, 'cast': [{'adult': False, 'gend...","{'id': 346372, 'results': {}}","{'id': 346372, 'keywords': [{'id': 2475, 'name..."
805484,346373,"{'adult': False, 'backdrop_path': None, 'belon...","{'id': 346373, 'cast': [], 'crew': [{'adult': ...","{'id': 346373, 'results': {'DE': {'link': 'htt...","{'id': 346373, 'keywords': [{'id': 10121, 'nam..."


In [ ]:
from time import time
t=time()
k=pd.DataFrame()
s=0
import numpy as np
for i in range(700000,len(d)):
  if not d.iloc[i]['movie'] is np.nan:
    x=eval(d.iloc[i]['movie'])
    y=eval(d.iloc[i]['credits'])
    x["cast"]=y["cast"]
    x["crew"]=y["crew"]
    x['keywords']=eval(d.iloc[i]['keys'])["keywords"]
    x['watch']=eval(d.iloc[i]['watch'])["results"]
    # display(y['cast'],y['crew'])
    k=pd.concat([k,pd.DataFrame([x])])
print("time taken :",time()-t)
k

time taken : 7049.445611000061


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,watch
0,False,/e6lPcvP0JoWfwceDg4cQqcOsyh.jpg,None,0,[],,540137,tt0273927,ml,Ormacheppu,...,Released,,Ormacheppu,False,0.0,0,"[{'adult': False, 'gender': 2, 'id': 182174, '...","[{'adult': False, 'gender': 2, 'id': 582196, '...",[],{}
0,False,None,None,0,[],,540138,tt0024177,fr,Incognito,...,Released,,Incognito,False,0.0,0,"[{'adult': False, 'gender': 2, 'id': 237765, '...","[{'adult': False, 'gender': 2, 'id': 2897, 'kn...",[],{}
0,False,None,None,0,[],,540139,tt0198556,fr,Il était une fois,...,Released,,Once Upon a Time,False,0.0,0,"[{'adult': False, 'gender': 1, 'id': 238605, '...","[{'adult': False, 'gender': 2, 'id': 1098119, ...",[],{}
0,False,None,None,0,[],,540140,tt0295344,fr,Idylle au Caire,...,Released,,Idylle au Caire,False,0.0,0,[],"[{'adult': False, 'gender': 2, 'id': 4344, 'kn...",[],{}
0,False,/gPct0NXJjukOba0LS8E8UwpdQbR.jpg,"{'id': 216545, 'name': 'City Hunter', 'poster_...",0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",http://cityhunter-movie.com/,540141,tt8161914,ja,劇場版シティーハンター ＜新宿プライベート・アイズ＞,...,Released,,City Hunter: Shinjuku Private Eyes,False,6.4,70,"[{'adult': False, 'gender': 2, 'id': 54690, 'k...","[{'adult': False, 'gender': 2, 'id': 16139, 'k...","[{'id': 13141, 'name': 'based on manga'}, {'id...",{'AE': {'link': 'https://www.themoviedb.org/mo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,False,None,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,346369,tt3531722,fr,T'es pas game,...,Released,,T'es pas game,False,6.0,1,"[{'adult': False, 'gender': 2, 'id': 1482013, ...","[{'adult': False, 'gender': 0, 'id': 1482012, ...",[],{}
0,False,None,None,0,"[{'id': 27, 'name': 'Horror'}]",,346371,tt2557172,tl,Zombie Island,...,Released,Armed with a slingshot.,The Grave Bandits,False,10.0,1,"[{'adult': False, 'gender': 0, 'id': 1482027, ...","[{'adult': False, 'gender': 0, 'id': 2384285, ...",[],{}
0,False,None,None,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",,346372,tt2977364,kk,Кыз-Жiбек,...,Released,,Lady Zhibek,False,8.0,1,"[{'adult': False, 'gender': 2, 'id': 1329275, ...","[{'adult': False, 'gender': 0, 'id': 1649165, ...","[{'id': 2475, 'name': 'kazakhstan'}, {'id': 37...",{}
0,False,None,None,0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",,346373,tt3831890,en,Whodunnit??,...,Released,,Whodunnit??,False,0.0,0,[],"[{'adult': False, 'gender': 0, 'id': 1482019, ...","[{'id': 10121, 'name': 'stop motion'}, {'id': ...",{'DE': {'link': 'https://www.themoviedb.org/mo...


In [ ]:
k.to_csv("/content/drive/MyDrive/mov7080.csv")

In [ ]:
def fn(a):
  b=eval(a)
  l=[]
  for i in b:
    l.append(i['name'])
  return(l)

In [ ]:
a['genres']=a['genres'].apply(fn)

In [ ]:
a=a.drop(['backdrop_path', 'belongs_to_collection','imdb_id',
       'original_language', 'original_title', 'overview', 'poster_path',
       'production_companies', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'imdbrat', 'imdbvote', 'gencode'],axis=1)

In [ ]:
d.to_parquet("temp3.brotli.parquet", compression='brotli')

In [ ]:
def get_movie(id):
    # print(request.args)
    a=""
    c=""
    k={}
    id=id
    print(d.index[0])
    print(type(d.index[0]))
    if id in d.index:
        k['id']=id
        if d.loc[id]['poster_path']==" ":
            k['poster']="https://"+o+"/files/poster.png"
        else:
            k['poster']="https://image.tmdb.org/t/p/original"+d.loc[id]['poster_path']
        k['title']=d.loc[id]['original_title']
        k['year']=d.loc[id]['release_date']
        k['rating']=d.loc[id]['vote_average']
        if d.loc[id]['backdrop_path']==" ":
            k['poster']="https://"+o+"/files/baground.png"
        else:
            k['poster']="https://image.tmdb.org/t/p/original"+d.loc[id]['backdrop_path']
        if d.loc[id]['belongs_to_collection']!=" ":
            k['collection']=eval(d.loc[id]['belongs_to_collection'])["name"]
        else:
            k['collection']="None"
        k['lang']=d.loc[id]['original_language']
        k['len']=d.loc[id]['runtime']
        k['status']=d.loc[id]['status']
        k['imdbrat']=d.loc[id]['imdbrat']
        k['imdbvote']=d.loc[id]['imdbvote']
        # print(d.loc[id].cast)
        i=eval(d.loc[id].cast)
        name=list()
        char=list()
        pop=list()
        a=[]
        c=[]
        if not i is None:
            for j in range(0,len(i)):
                b={}
                b['name']=i[j]['name']
                b['character']=i[j]['character']
                b['popularity']=i[j]['popularity']
                if i[j]['profile_path'] is not None:
                    b['pic']="https://image.tmdb.org/t/p/original"+i[j]['profile_path']
                else:
                    b['pic']="https://"+o+"/files"+str(i[j]['gender'])+".png"
                a.append(b)
            a=sorted(a,key=lambda x:x['popularity'],reverse=True)
        j=eval(d.loc[id]['crew'])
        if not j is None:
            for z in range(0,len(j)):
                b={}
                b['name']=j[z]['name']
                b['job']=j[z]['job']
                if "director" in b['job'].lower():
                    b['pop']=1
                elif "producer" in b['job'].lower():
                    b['pop']=2
                else:
                    b['pop']=3
                if j[z]['profile_path'] is not None:
                    b['pic']="https://image.tmdb.org/t/p/original"+j[z]['profile_path']
                else:
                    b['pic']="abc"+"/files"+str(j[z]['gender'])+".png"
                c.append(b)
            c=sorted(c,key=lambda x:x['pop'])
        k['cast']=a
        k['crew']=c
    else:
        return("Wrong ID")
    print(k)
    return jsonify(k)

In [ ]:
get_movie(550)

393209
<class 'numpy.int64'>
{'id': 550, 'poster': 'https://image.tmdb.org/t/p/original/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg', 'title': 'Fight Club', 'year': Timestamp('1999-10-15 00:00:00'), 'rating': 8.428, 'cast': [{'name': 'Brad Pitt', 'character': 'Tyler Durden', 'popularity': 66.422, 'pic': 'https://image.tmdb.org/t/p/original/oTB9vGIBacH5aQNS0pUM74QSWuf.jpg'}, {'name': 'Helena Bonham Carter', 'character': 'Marla Singer', 'popularity': 37.444, 'pic': 'https://image.tmdb.org/t/p/original/DDeITcCpnBd0CkAIRPhggy9bt5.jpg'}, {'name': 'Jared Leto', 'character': 'Angel Face', 'popularity': 32.857, 'pic': 'https://image.tmdb.org/t/p/original/i6zIpHiKBnaHtacJU6qgeoDjiw1.jpg'}, {'name': 'Edward Norton', 'character': 'The Narrator', 'popularity': 21.045, 'pic': 'https://image.tmdb.org/t/p/original/5XBzD5WuTyVQZeS4VI25z2moMeY.jpg'}, {'name': 'Holt McCallany', 'character': 'The Mechanic', 'popularity': 12.231, 'pic': 'https://image.tmdb.org/t/p/original/a5ax2ICLrV6l0T74OSFvzssCQyQ.jpg'}, {'name': 

RuntimeError: ignored

In [ ]:
print(hello)

NameError: ignored

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 17.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=75e6119529308f06225084189527ca41f2c3c7960af8c607236db9a44f8d7374
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [ ]:
from flask import send_from_directory,url_for,redirect,request
from flask import Flask
from urllib.parse import urlparse
from flask_ngrok import run_with_ngrok as rn

# Flask constructor takes the name of
# current module (__name__) as argument.
app = Flask(__name__)
rn(app)
# The route() function of the Flask class is a decorator,
# which tells the application which URL should call
# the associated function.
@app.route('/')
# ‘/’ URL is bound with hello_world() function.
def hello_world():
    o = urlparse(request.base_url)
    print(o.netloc)
    return 'Hello World'

@app.route('/files/<path:filename>')
def download_file(filename):
    print(filename)
    return send_from_directory("./files/",filename, as_attachment=False)

# main driver function
if __name__ == '__main__':

    # run() method of Flask class runs the application
    # on the local development server.
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0b9a-35-221-14-97.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 18:52:51] "GET / HTTP/1.1" 200 -


0b9a-35-221-14-97.ngrok.io


...

baground.png


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 18:53:25] "GET /files/baground.jpg HTTP/1.1" 200 -


baground.jpg


INFO:werkzeug:127.0.0.1 - - [02/Jan/2023 18:54:58] "GET /files/baground.jpg HTTP/1.1" 200 -


baground.jpg
